In [1]:
from rapidz import Stream

In [2]:
from time import sleep
import operator as op

def sleepy_add(x):
    sleep(2)
    return op.add(x, 1)

In [3]:
source = Stream()
source.map(sleepy_add).zip('serial').sink(print)

Output()

In [4]:
for i in range(5):
    source.emit(i)

(1, 'serial')
(2, 'serial')
(3, 'serial')
(4, 'serial')
(5, 'serial')


In [5]:
source2 = Stream()
source2.scatter(backend='thread').map(sleepy_add).buffer(10).gather().zip('parallel').sink(print)

Output()

In [6]:
for i in range(5):
    source2.emit(i)

(1, 'parallel')
(2, 'parallel')
(3, 'parallel')
(4, 'parallel')
(5, 'parallel')


In [7]:
from dask.distributed import Client
client = Client(processes=False)
# open http://localhost:8787/status

In [8]:
source3 = Stream()
source3.scatter().map(sleepy_add).buffer(10).gather().zip('parallel').sink(print)

Output()

In [9]:
for i in range(5):
    source3.emit(i)

(1, 'parallel')
(2, 'parallel')
(3, 'parallel')
(4, 'parallel')
(5, 'parallel')


In [10]:
# Many of the existing nodes work! (in rapidz)
source3 = Stream()
source3.scatter().map(sleepy_add).filter(lambda x: x%2==0).buffer(10).gather().zip('parallel').sink(print)
for i in range(5):
    source3.emit(i)

(2, 'parallel')
(4, 'parallel')


In [11]:
# joining is problematic (see https://github.com/xpdAcq/rapidz/pull/20 for active work on this)
source3 = Stream()
filtered_parallel = source3.map(sleepy_add).filter(lambda x: x%2==0)
filtered_parallel.combine_latest(source3, emit_on=0).zip('serial').sink(print)
for i in range(5):
    source3.emit(i)

source3 = Stream()
filtered_parallel = source3.scatter().map(sleepy_add).filter(lambda x: x%2==0)
filtered_parallel.combine_latest(source3, emit_on=0).buffer(10).gather().zip('parallel').sink(print)
for i in range(5):
    source3.emit(i)

((2, 0), 'serial')
((4, 2), 'serial')
((2, 1), 'parallel')
((4, 3), 'parallel')
